# Lab 2-3 (추가): KServe Canary 배포 실습

## 📋 실습 개요

| 항목 | 내용 |
|------|------|
| **소요시간** | 40분 |
| **난이도** | ⭐⭐⭐⭐ |
| **목표** | KServe Canary 배포를 통한 점진적 모델 업데이트 |
| **KServe 버전** | v0.15.x |

---

## 🎯 KServe Canary 배포 원리

1. `storageUri`를 변경하면 새로운 **Revision**이 자동 생성됨
2. `canaryTrafficPercent`로 새 Revision에 보낼 트래픽 비율 설정
3. 검증 후 `canaryTrafficPercent`를 제거하면 새 Revision이 100% 트래픽 수신

```
┌─────────────────────────────────────────────────────────────┐
│                    Canary 배포 흐름                          │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  Step 1: storageUri 변경 + canaryTrafficPercent: 20         │
│  ┌──────────────────────────────────────────┐               │
│  │ Revision-001 (80%)  ████████████████░░░░  │               │
│  │ Revision-002 (20%)  █████░░░░░░░░░░░░░░░  │  ← 새 모델    │
│  └──────────────────────────────────────────┘               │
│                         ↓                                   │
│  Step 2: canaryTrafficPercent: 50                           │
│  ┌──────────────────────────────────────────┐               │
│  │ Revision-001 (50%)  █████████████░░░░░░░  │               │
│  │ Revision-002 (50%)  █████████████░░░░░░░  │               │
│  └──────────────────────────────────────────┘               │
│                         ↓                                   │
│  Step 3: canaryTrafficPercent 제거 (승격)                    │
│  ┌──────────────────────────────────────────┐               │
│  │ Revision-001 (0%)   ░░░░░░░░░░░░░░░░░░░░  │               │
│  │ Revision-002 (100%) █████████████████████  │  ← 새 Default│
│  └──────────────────────────────────────────┘               │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

## 1. 환경 설정

In [ ]:
# ============================================================
# 1.1 라이브러리 Import
# ============================================================
from kubernetes import client, config
from kubernetes.client.rest import ApiException
import requests
import json
import time
import random
import os
import base64
import boto3
from botocore.exceptions import ClientError, NoCredentialsError

print("✅ 라이브러리 Import 완료")

In [ ]:
# ============================================================
# 1.2 Kubernetes 클라이언트 설정
# ============================================================

# In-cluster 설정 로드
config.load_incluster_config()

# API 클라이언트 생성
core_v1 = client.CoreV1Api()
custom_api = client.CustomObjectsApi()

print("✅ Kubernetes 클라이언트 연결 완료")

In [ ]:
# ============================================================
# 1.3 네임스페이스 및 환경 변수 설정
# ============================================================

# 네임스페이스 자동 감지
def get_current_namespace():
    """Kubeflow Notebook의 현재 네임스페이스 자동 감지"""
    try:
        with open('/var/run/secrets/kubernetes.io/serviceaccount/namespace', 'r') as f:
            return f.read().strip()
    except FileNotFoundError:
        return "kubeflow-user01"  # 기본값

NAMESPACE = get_current_namespace()

# 사용자 번호 추출
USER_NUM = NAMESPACE.replace("kubeflow-user", "")

# 모델 이름 (네임스페이스에 맞게 자동 설정)
MODEL_NAME = f"california-model-user{USER_NUM}"

# S3 버킷
S3_BUCKET = f"mlops-training-user{USER_NUM}"

# AWS 리전
AWS_REGION = "ap-northeast-2"

print("="*60)
print("  Canary 배포 환경 설정")
print("="*60)
print(f"  📁 네임스페이스: {NAMESPACE}")
print(f"  👤 사용자 번호: {USER_NUM}")
print(f"  📦 모델명: {MODEL_NAME}")
print(f"  🗂️  S3 버킷: {S3_BUCKET}")
print(f"  🌏 AWS 리전: {AWS_REGION}")
print("="*60)

In [ ]:
# ============================================================
# 1.4 ⚠️ AWS 자격증명 설정 (Kubernetes Secret에서 로드)
# ============================================================
print("="*60)
print("  AWS 자격증명 설정")
print("="*60)

def get_aws_credentials_from_secret(namespace):
    """Kubernetes Secret에서 AWS 자격증명 읽기"""
    try:
        secret = core_v1.read_namespaced_secret("aws-s3-credentials", namespace)
        
        access_key = base64.b64decode(secret.data["AWS_ACCESS_KEY_ID"]).decode("utf-8")
        secret_key = base64.b64decode(secret.data["AWS_SECRET_ACCESS_KEY"]).decode("utf-8")
        
        # 환경 변수에 설정
        os.environ["AWS_ACCESS_KEY_ID"] = access_key
        os.environ["AWS_SECRET_ACCESS_KEY"] = secret_key
        os.environ["AWS_DEFAULT_REGION"] = AWS_REGION
        
        print(f"  ✅ AWS 자격증명 로드 완료 (from Secret 'aws-s3-credentials')")
        print(f"     - Access Key: {access_key[:8]}...{access_key[-4:]}")
        print(f"     - Region: {AWS_REGION}")
        return True
        
    except ApiException as e:
        if e.status == 404:
            print(f"  ❌ Secret 'aws-s3-credentials'를 찾을 수 없습니다.")
            print(f"     아래 셀에서 수동으로 입력해주세요.")
        else:
            print(f"  ❌ Secret 조회 실패: {e.reason}")
        return False

# Secret에서 자격증명 로드 시도
credentials_loaded = get_aws_credentials_from_secret(NAMESPACE)

In [ ]:
# ============================================================
# 1.5 AWS 자격증명 수동 설정 (Secret이 없는 경우)
# ============================================================

# ⚠️ Secret에서 자격증명을 로드하지 못한 경우에만 아래 코드를 수정하고 실행하세요!

if not credentials_loaded:
    print("  ⚠️ AWS 자격증명을 수동으로 설정합니다.")
    print("     아래 값들을 본인의 자격증명으로 수정하세요!")
    print("")
    
    # ⚠️ 본인의 AWS 자격증명으로 수정하세요!
    AWS_ACCESS_KEY_ID = "YOUR_ACCESS_KEY_HERE"      # 수정 필요!
    AWS_SECRET_ACCESS_KEY = "YOUR_SECRET_KEY_HERE"  # 수정 필요!
    
    if AWS_ACCESS_KEY_ID == "YOUR_ACCESS_KEY_HERE":
        print("  ❌ AWS 자격증명을 수정하지 않았습니다!")
        print("     위의 값들을 본인의 자격증명으로 수정해주세요.")
    else:
        os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
        os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
        os.environ["AWS_DEFAULT_REGION"] = AWS_REGION
        print(f"  ✅ AWS 자격증명 설정 완료")
        credentials_loaded = True
else:
    print("  ℹ️ AWS 자격증명이 이미 Secret에서 로드되었습니다.")

In [ ]:
# ============================================================
# 1.6 AWS 자격증명 검증 (boto3 사용)
# ============================================================
print("="*60)
print("  AWS 자격증명 검증")
print("="*60)

def verify_s3_access(bucket_name, region):
    """boto3를 사용하여 S3 버킷 접근 테스트"""
    try:
        s3_client = boto3.client('s3', region_name=region)
        
        # 버킷 내용 조회 (최대 5개)
        response = s3_client.list_objects_v2(
            Bucket=bucket_name,
            MaxKeys=5
        )
        
        print(f"  ✅ S3 버킷 '{bucket_name}' 접근 성공!")
        print(f"")
        
        if 'Contents' in response:
            print(f"  📂 버킷 내용 (일부):")
            for obj in response['Contents'][:5]:
                print(f"     - {obj['Key']}")
        else:
            print(f"  📂 버킷이 비어있거나 root에 객체가 없습니다.")
            
        return True
        
    except NoCredentialsError:
        print(f"  ❌ AWS 자격증명이 설정되지 않았습니다.")
        return False
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'NoSuchBucket':
            print(f"  ❌ 버킷 '{bucket_name}'이 존재하지 않습니다.")
        elif error_code == 'AccessDenied':
            print(f"  ❌ 버킷 '{bucket_name}'에 접근 권한이 없습니다.")
        else:
            print(f"  ❌ S3 오류: {e}")
        return False

# S3 접근 테스트
s3_accessible = verify_s3_access(S3_BUCKET, AWS_REGION)

---

## 2. 현재 InferenceService 상태 확인

In [ ]:
# ============================================================
# 2.1 현재 InferenceService 상태 확인
# ============================================================
print("="*60)
print("  현재 InferenceService 상태 확인")
print("="*60)

def get_inferenceservice_details(model_name, namespace):
    """InferenceService 상세 정보 조회"""
    try:
        isvc = custom_api.get_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        
        # 기본 정보
        url = isvc.get("status", {}).get("url", "N/A")
        conditions = isvc.get("status", {}).get("conditions", [])
        ready = next((c for c in conditions if c.get("type") == "Ready"), {})
        
        # 현재 storageUri
        storage_uri = isvc.get("spec", {}).get("predictor", {}).get("model", {}).get("storageUri", "N/A")
        
        # Canary 트래픽 비율
        canary_percent = isvc.get("spec", {}).get("predictor", {}).get("canaryTrafficPercent", None)
        
        # 트래픽 정보
        traffic = isvc.get("status", {}).get("components", {}).get("predictor", {}).get("traffic", [])
        latest_ready = isvc.get("status", {}).get("components", {}).get("predictor", {}).get("latestReadyRevision", "N/A")
        
        print(f"  📊 모델명: {model_name}")
        print(f"  📁 네임스페이스: {namespace}")
        print(f"  🔗 URL: {url}")
        print(f"  ✅ Ready: {ready.get('status', 'Unknown')}")
        print(f"  📦 Storage URI: {storage_uri}")
        print(f"  🎯 Canary Traffic %: {canary_percent if canary_percent else 'Not set (100% to latest)'}")
        print(f"  📝 Latest Ready Revision: {latest_ready}")
        
        if traffic:
            print(f"\n  📊 트래픽 분배:")
            for t in traffic:
                revision = t.get("revisionName", "unknown")
                percent = t.get("percent", 0)
                latest = t.get("latestRevision", False)
                tag = " (latest)" if latest else ""
                print(f"     - {revision}: {percent}%{tag}")
        
        return isvc
        
    except ApiException as e:
        if e.status == 404:
            print(f"  ❌ InferenceService '{model_name}'를 찾을 수 없습니다.")
            print(f"     Lab 2-3 기본 실습을 먼저 완료해주세요!")
        else:
            print(f"  ❌ 조회 실패: {e.reason}")
        return None

current_isvc = get_inferenceservice_details(MODEL_NAME, NAMESPACE)

In [ ]:
# ============================================================
# 2.2 현재 Storage URI 저장 (V1)
# ============================================================
print("="*60)
print("  현재 모델 경로 저장 (V1)")
print("="*60)

if current_isvc:
    STORAGE_URI_V1 = current_isvc.get("spec", {}).get("predictor", {}).get("model", {}).get("storageUri", "")
    print(f"  📦 V1 Storage URI: {STORAGE_URI_V1}")
    
    # Run ID 추출
    if "mlflow-artifacts/" in STORAGE_URI_V1:
        parts = STORAGE_URI_V1.split("mlflow-artifacts/")[1].split("/")
        RUN_ID_V1 = parts[0]
        print(f"  🔑 V1 Run ID: {RUN_ID_V1}")
    else:
        RUN_ID_V1 = ""
else:
    STORAGE_URI_V1 = ""
    RUN_ID_V1 = ""
    print("  ❌ InferenceService를 찾을 수 없습니다.")

---

## 3. V2 모델 준비 (새 모델 학습)

Canary 배포를 위해서는 **새로운 모델 (V2)**이 필요합니다.  
아래에서 새 모델을 학습하고 MLflow에 등록합니다.

In [ ]:
# ============================================================
# 3.1 MLflow 설정
# ============================================================
print("="*60)
print("  MLflow 설정")
print("="*60)

import mlflow

# MLflow 서버 설정
MLFLOW_TRACKING_URI = "http://mlflow-server:5000"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# S3 artifact 위치 설정을 위한 환경변수 (이미 설정됨)
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"https://s3.{AWS_REGION}.amazonaws.com"

# 실험 설정
EXPERIMENT_NAME = f"california-housing-user{USER_NUM}"
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"  📊 MLflow Tracking URI: {MLFLOW_TRACKING_URI}")
print(f"  📁 Experiment: {EXPERIMENT_NAME}")
print(f"  🗂️  Artifact Location: s3://{S3_BUCKET}/mlflow-artifacts")

In [ ]:
# ============================================================
# 3.2 V2 모델 학습 및 MLflow 등록
# ============================================================
print("="*60)
print("  V2 모델 학습 및 MLflow 등록")
print("="*60)

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 데이터 로드
print("  📥 데이터 로드 중...")
data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.2, random_state=42
)

# V2 모델 학습 (다른 하이퍼파라미터)
print("  🔄 V2 모델 학습 중 (n_estimators=150, max_depth=15)...")

with mlflow.start_run(run_name=f"canary-v2-model-user{USER_NUM}") as run:
    # 하이퍼파라미터 (V1과 다른 값 사용)
    params = {
        "n_estimators": 150,  # V1은 100이었다고 가정
        "max_depth": 15,      # V1은 10이었다고 가정
        "random_state": 42,
        "model_version": "v2-canary"
    }
    
    # 파라미터 로깅
    mlflow.log_params(params)
    
    # 모델 학습
    model = RandomForestRegressor(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        random_state=params["random_state"]
    )
    model.fit(X_train, y_train)
    
    # 예측 및 메트릭
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    # 메트릭 로깅
    mlflow.log_metrics({"mse": mse, "rmse": rmse, "r2": r2})
    
    # 모델 저장
    mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name=f"california-housing-rf-user{USER_NUM}"
    )
    
    RUN_ID_V2 = run.info.run_id
    
    print(f"")
    print(f"  ✅ V2 모델 학습 완료!")
    print(f"     Run ID: {RUN_ID_V2}")
    print(f"     MSE: {mse:.4f}")
    print(f"     RMSE: {rmse:.4f}")
    print(f"     R²: {r2:.4f}")

# V2 Storage URI 구성
STORAGE_URI_V2 = f"s3://{S3_BUCKET}/mlflow-artifacts/{RUN_ID_V2}/artifacts/model"
print(f"")
print(f"  📦 V2 Storage URI: {STORAGE_URI_V2}")

In [ ]:
# ============================================================
# 3.3 S3에 모델 업로드 확인 (boto3 사용)
# ============================================================
print("="*60)
print("  S3 모델 경로 확인")
print("="*60)

# 잠시 대기 (S3 업로드 완료 대기)
print("  ⏳ S3 업로드 완료 대기 (5초)...")
time.sleep(5)

def verify_model_in_s3(bucket, run_id, region):
    """S3에서 모델 파일 확인"""
    try:
        s3_client = boto3.client('s3', region_name=region)
        prefix = f"mlflow-artifacts/{run_id}/artifacts/model/"
        
        response = s3_client.list_objects_v2(
            Bucket=bucket,
            Prefix=prefix,
            MaxKeys=10
        )
        
        if 'Contents' in response and len(response['Contents']) > 0:
            print(f"  ✅ S3에 V2 모델 파일 존재:")
            for obj in response['Contents']:
                key = obj['Key'].replace(prefix, '')
                if key:  # 빈 문자열 제외
                    print(f"     - {key}")
            return True
        else:
            print(f"  ❌ 모델 파일을 찾을 수 없습니다.")
            print(f"     경로: s3://{bucket}/{prefix}")
            return False
            
    except ClientError as e:
        print(f"  ❌ S3 확인 실패: {e}")
        return False

# 모델 파일 확인
model_exists = verify_model_in_s3(S3_BUCKET, RUN_ID_V2, AWS_REGION)

---

## 4. Canary 배포 적용 (20% 트래픽)

**KServe v0.11+ Canary 배포 방식:**
- `storageUri`를 새 모델 경로로 변경
- `canaryTrafficPercent`로 새 Revision에 보낼 트래픽 비율 설정

In [ ]:
# ============================================================
# 4.1 V1 / V2 Storage URI 확인
# ============================================================
print("="*60)
print("  Canary 배포 준비")
print("="*60)

print(f"  📦 V1 (현재): {STORAGE_URI_V1}")
print(f"  📦 V2 (Canary): {STORAGE_URI_V2}")
print("")

if not STORAGE_URI_V1:
    print("  ⚠️ V1 Storage URI가 설정되지 않았습니다.")
    print("     현재 InferenceService 상태를 확인해주세요.")
    
if not STORAGE_URI_V2:
    print("  ⚠️ V2 Storage URI가 설정되지 않았습니다.")
    print("     V2 모델 학습을 먼저 완료해주세요.")

In [ ]:
# ============================================================
# 4.2 Canary 배포 스펙 정의 (20% 트래픽)
# ============================================================
print("="*60)
print("  Step 1: Canary 배포 (20% 트래픽)")
print("="*60)

CANARY_TRAFFIC_PERCENT = 20

# KServe v0.15 호환 Canary 스펙
canary_isvc_spec = {
    "apiVersion": "serving.kserve.io/v1beta1",
    "kind": "InferenceService",
    "metadata": {
        "name": MODEL_NAME,
        "namespace": NAMESPACE,
        "annotations": {
            "sidecar.istio.io/inject": "false"
        }
    },
    "spec": {
        "predictor": {
            # ⚡ Canary 트래픽 비율 설정
            "canaryTrafficPercent": CANARY_TRAFFIC_PERCENT,
            "minReplicas": 1,
            "maxReplicas": 1,
            "model": {
                "modelFormat": {"name": "sklearn"},
                # ⚡ 새 모델 경로 (V2) - storageUri 변경으로 새 Revision 생성
                "storageUri": STORAGE_URI_V2,
                "resources": {
                    "requests": {"cpu": "200m", "memory": "512Mi"},
                    "limits": {"cpu": "500m", "memory": "1Gi"}
                }
            }
        }
    }
}

print(f"  📝 Canary 설정:")
print(f"     - 기존 Revision: {100 - CANARY_TRAFFIC_PERCENT}% 트래픽")
print(f"     - 새 Revision (V2): {CANARY_TRAFFIC_PERCENT}% 트래픽")
print(f"     - V2 Storage URI: {STORAGE_URI_V2}")

In [ ]:
# ============================================================
# 4.3 Canary 배포 적용
# ============================================================
print("="*60)
print("  Canary 배포 적용 중...")
print("="*60)

def apply_inferenceservice(spec, namespace):
    """InferenceService 생성 또는 업데이트"""
    model_name = spec["metadata"]["name"]
    
    try:
        # 기존 InferenceService 조회
        existing = custom_api.get_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        
        # resourceVersion 복사 (업데이트용)
        spec["metadata"]["resourceVersion"] = existing["metadata"]["resourceVersion"]
        
        # 업데이트
        custom_api.replace_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name,
            body=spec
        )
        print(f"  ✅ InferenceService '{model_name}' 업데이트 완료")
        
    except ApiException as e:
        if e.status == 404:
            # 새로 생성
            custom_api.create_namespaced_custom_object(
                group="serving.kserve.io",
                version="v1beta1",
                namespace=namespace,
                plural="inferenceservices",
                body=spec
            )
            print(f"  ✅ InferenceService '{model_name}' 생성 완료")
        else:
            print(f"  ❌ 오류: {e.reason}")
            raise

# Canary 배포 적용
apply_inferenceservice(canary_isvc_spec, NAMESPACE)

In [ ]:
# ============================================================
# 4.4 Canary 배포 상태 모니터링
# ============================================================
print("="*60)
print("  Canary 배포 상태 모니터링")
print("="*60)

def wait_for_canary_ready(model_name, namespace, timeout=300):
    """Canary 배포 완료 대기"""
    print(f"  ⏳ 새 Revision Ready 대기 중...")
    print(f"     (최대 {timeout}초, 보통 2-4분 소요)")
    print("")
    
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            isvc = custom_api.get_namespaced_custom_object(
                group="serving.kserve.io",
                version="v1beta1",
                namespace=namespace,
                plural="inferenceservices",
                name=model_name
            )
            
            conditions = isvc.get("status", {}).get("conditions", [])
            ready = next((c for c in conditions if c.get("type") == "Ready"), None)
            
            # 트래픽 상태 확인
            traffic = isvc.get("status", {}).get("components", {}).get("predictor", {}).get("traffic", [])
            
            elapsed = int(time.time() - start_time)
            
            if ready:
                status = ready.get("status")
                reason = ready.get("reason", "")
                
                if status == "True":
                    print(f"\n  ✅ Canary 배포 Ready! ({elapsed}초 소요)")
                    print(f"\n  📊 트래픽 분배 상태:")
                    for t in traffic:
                        revision = t.get("revisionName", "unknown")
                        percent = t.get("percent", 0)
                        latest = t.get("latestRevision", False)
                        tag = " (Canary - V2)" if latest else " (Default - V1)"
                        print(f"     - {revision}: {percent}%{tag}")
                    return True
                else:
                    print(f"  ⏳ Status: {status} | Reason: {reason} ({elapsed}초)")
            
        except ApiException as e:
            print(f"  ⚠️ 상태 확인 실패: {e.reason}")
        
        time.sleep(15)
    
    print(f"\n  ❌ 타임아웃: {timeout}초 초과")
    return False

# 모니터링 실행
is_canary_ready = wait_for_canary_ready(MODEL_NAME, NAMESPACE)

---

## 5. 트래픽 분할 검증

In [ ]:
# ============================================================
# 5.1 추론 테스트 함수 정의
# ============================================================
print("="*60)
print("  트래픽 분할 검증 준비")
print("="*60)

# California Housing 테스트 데이터 (8개 특성)
TEST_SAMPLES = [
    [8.3252, 41.0, 6.984, 1.023, 322.0, 2.555, 37.88, -122.23],
    [3.5, 25.0, 5.5, 1.1, 1500.0, 3.0, 37.5, -122.0],
    [5.0, 30.0, 6.0, 1.0, 1000.0, 2.8, 34.0, -118.0],
]

def test_inference(model_name, namespace, data):
    """클러스터 내부에서 추론 테스트"""
    url = f"http://{model_name}-predictor.{namespace}.svc.cluster.local/v1/models/{model_name}:predict"
    
    payload = {"instances": [data]}
    
    try:
        response = requests.post(
            url,
            json=payload,
            headers={"Content-Type": "application/json"},
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            prediction = result.get("predictions", [None])[0]
            return {"success": True, "prediction": prediction}
        else:
            return {"success": False, "error": response.text}
            
    except Exception as e:
        return {"success": False, "error": str(e)}

print("  ✅ 추론 테스트 함수 정의 완료")

In [ ]:
# ============================================================
# 5.2 다수 요청으로 트래픽 분할 검증
# ============================================================
print("="*60)
print("  트래픽 분할 검증 (30회 요청)")
print("="*60)

def verify_traffic(model_name, namespace, num_requests=30):
    """다수의 요청을 보내 트래픽 분할 확인"""
    print(f"  📡 {num_requests}개 요청 전송 중...")
    print("")
    
    predictions = []
    success_count = 0
    
    for i in range(num_requests):
        test_data = random.choice(TEST_SAMPLES)
        result = test_inference(model_name, namespace, test_data)
        
        if result["success"]:
            predictions.append(round(result["prediction"], 4))
            success_count += 1
        
        if (i + 1) % 10 == 0:
            print(f"     진행: {i + 1}/{num_requests}")
    
    print("")
    print(f"  📊 결과:")
    print(f"     - 성공: {success_count}/{num_requests}")
    
    if predictions:
        unique_predictions = len(set(predictions))
        print(f"     - 고유 예측값 수: {unique_predictions}")
        print(f"     - 예측값 범위: {min(predictions):.4f} ~ {max(predictions):.4f}")
    
    return predictions

# 트래픽 검증
predictions = verify_traffic(MODEL_NAME, NAMESPACE, 30)

---

## 6. Canary 트래픽 증가 (50%)

In [ ]:
# ============================================================
# 6.1 Canary 트래픽 50%로 증가
# ============================================================
print("="*60)
print("  Step 2: Canary 트래픽 50%로 증가")
print("="*60)

def update_canary_traffic(model_name, namespace, traffic_percent):
    """Canary 트래픽 비율 업데이트 (KServe v0.15 방식)"""
    try:
        # 현재 InferenceService 조회
        isvc = custom_api.get_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        
        # canaryTrafficPercent 업데이트
        if "predictor" in isvc.get("spec", {}):
            isvc["spec"]["predictor"]["canaryTrafficPercent"] = traffic_percent
        else:
            print(f"  ⚠️ Predictor가 설정되어 있지 않습니다.")
            return False
        
        # 업데이트 적용
        custom_api.replace_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name,
            body=isvc
        )
        
        print(f"  ✅ Canary 트래픽을 {traffic_percent}%로 업데이트 완료!")
        print(f"     - 기존 Revision (V1): {100 - traffic_percent}%")
        print(f"     - 새 Revision (V2): {traffic_percent}%")
        return True
        
    except ApiException as e:
        print(f"  ❌ 업데이트 실패: {e.reason}")
        return False

# 50%로 업데이트
update_canary_traffic(MODEL_NAME, NAMESPACE, 50)

In [ ]:
# ============================================================
# 6.2 업데이트 확인
# ============================================================
print("="*60)
print("  트래픽 업데이트 확인")
print("="*60)

print("  ⏳ 30초 대기 중...")
time.sleep(30)

get_inferenceservice_details(MODEL_NAME, NAMESPACE)

---

## 7. 롤백 전략 (선택 실습)

In [ ]:
# ============================================================
# 7.1 롤백 함수 정의
# ============================================================
print("="*60)
print("  롤백 전략 (참고용)")
print("="*60)

def rollback_canary(model_name, namespace, original_storage_uri):
    """Canary 롤백 - 기존 모델로 복원"""
    print("  🔄 Canary 롤백 시작...")
    
    try:
        isvc = custom_api.get_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        
        # canaryTrafficPercent 제거 + 원래 storageUri로 복원
        if "canaryTrafficPercent" in isvc.get("spec", {}).get("predictor", {}):
            del isvc["spec"]["predictor"]["canaryTrafficPercent"]
        
        isvc["spec"]["predictor"]["model"]["storageUri"] = original_storage_uri
        
        custom_api.replace_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name,
            body=isvc
        )
        
        print(f"  ✅ 롤백 완료! 기존 모델(V1)로 복원됨")
        print(f"     Storage URI: {original_storage_uri}")
        return True
        
    except ApiException as e:
        print(f"  ❌ 롤백 실패: {e.reason}")
        return False

print("")
print("  📝 롤백 방법:")
print(f"     rollback_canary('{MODEL_NAME}', '{NAMESPACE}', '{STORAGE_URI_V1}')")
print("")
print("  ⚠️ 롤백이 필요하면 아래 셀의 주석을 해제하고 실행하세요.")

In [ ]:
# ============================================================
# 7.2 롤백 실행 (필요시 주석 해제)
# ============================================================

# ⚠️ 롤백이 필요하면 아래 주석을 해제하고 실행하세요!
# rollback_canary(MODEL_NAME, NAMESPACE, STORAGE_URI_V1)

print("  ℹ️ 롤백 셀입니다. 필요시 위 주석을 해제하고 실행하세요.")

---

## 8. Canary 승격 (100% 트래픽)

In [ ]:
# ============================================================
# 8.1 Canary 승격 (canaryTrafficPercent 제거)
# ============================================================
print("="*60)
print("  Step 3: Canary 승격 (100% 트래픽)")
print("="*60)

def promote_canary(model_name, namespace):
    """Canary를 새로운 Default로 승격 (canaryTrafficPercent 제거)"""
    print("  🚀 Canary 승격 중...")
    
    try:
        isvc = custom_api.get_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        
        # canaryTrafficPercent 제거 → 새 Revision이 100% 트래픽
        if "canaryTrafficPercent" in isvc.get("spec", {}).get("predictor", {}):
            del isvc["spec"]["predictor"]["canaryTrafficPercent"]
            
            custom_api.replace_namespaced_custom_object(
                group="serving.kserve.io",
                version="v1beta1",
                namespace=namespace,
                plural="inferenceservices",
                name=model_name,
                body=isvc
            )
            
            print(f"  ✅ Canary 승격 완료!")
            print(f"     - 새 모델 (V2)이 100% 트래픽을 받습니다")
            return True
        else:
            print(f"  ⚠️ canaryTrafficPercent가 이미 설정되어 있지 않습니다.")
            print(f"     이미 승격되었거나, Canary 배포가 활성화되지 않았습니다.")
            return False
        
    except ApiException as e:
        print(f"  ❌ 승격 실패: {e.reason}")
        return False

# Canary 승격
promote_canary(MODEL_NAME, NAMESPACE)

In [ ]:
# ============================================================
# 8.2 승격 완료 확인
# ============================================================
print("="*60)
print("  승격 완료 확인")
print("="*60)

print("  ⏳ 30초 대기 중...")
time.sleep(30)

get_inferenceservice_details(MODEL_NAME, NAMESPACE)

---

## 9. 실습 완료 체크리스트

In [ ]:
# ============================================================
# 9. 실습 완료 체크리스트
# ============================================================
print("="*60)
print("  Lab 2-3 (Canary 배포) 완료 체크리스트")
print("="*60)
print("")
print("  ✅ 1. KServe Canary 배포 원리 이해")
print("      - storageUri 변경 → 새 Revision 생성")
print("      - canaryTrafficPercent로 트래픽 분할")
print("")
print("  ✅ 2. V2 모델 학습 및 MLflow 등록")
print("      - 새 하이퍼파라미터로 모델 학습")
print("      - S3에 모델 저장")
print("")
print("  ✅ 3. Canary 배포 적용")
print("      - 20% 트래픽으로 시작")
print("      - 50%로 증가")
print("")
print("  ✅ 4. 롤백 전략")
print("      - 원래 storageUri로 복원")
print("")
print("  ✅ 5. Canary 승격")
print("      - canaryTrafficPercent 제거")
print("      - 새 모델이 100% 트래픽")
print("")
print("="*60)
print("  🎉 Canary 배포 실습 완료!")
print("="*60)

---

## 📚 핵심 요약 (KServe v0.15)

```yaml
# Canary 배포 YAML
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
spec:
  predictor:
    canaryTrafficPercent: 20  # 새 Revision에 20%
    model:
      storageUri: "s3://bucket/path/to/v2/model"  # 새 모델 경로
```

```python
# 트래픽 비율 조정
isvc["spec"]["predictor"]["canaryTrafficPercent"] = 50

# 승격 (canaryTrafficPercent 제거)
del isvc["spec"]["predictor"]["canaryTrafficPercent"]

# 롤백 (원래 storageUri로 복원 + canaryTrafficPercent 제거)
del isvc["spec"]["predictor"]["canaryTrafficPercent"]
isvc["spec"]["predictor"]["model"]["storageUri"] = original_uri
```

---

© 2025 현대오토에버 MLOps Training